## <b><font color='darkblue'>Preface</font></b>
([source](https://learn.deeplearning.ai/courses/langchain/lesson/3/memory)) <b><font size='3ptx'>From this chapter, we are going to see how we could use memory to let chatbot remember the chatting history.</font></b>

<a id='sect0'></a>
### <b><font color='darkgreen'>Outline</font></b>
* <b><font size='3ptx'><a href='#sect1'>ConversationBufferMemory</a></font></b>
* <b><font size='3ptx'><a href='#sect2'>ConversationBufferWindowMemory</a></font></b>
* <b><font size='3ptx'><a href='#sect3'>ConversationTokenBufferMemory</a></font></b>
* <b><font size='3ptx'><a href='#sect4'>ConversationSummaryMemory</a></font></b>

In [7]:
import datetime
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

<a id='sect1'></a>
## <b><font color='darkblue'>ConversationBufferMemory</font></b>
<b><font size='3ptx'>This memory [ConversationBufferMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer.ConversationBufferMemory.html#langchain-memory-buffer-conversationbuffermemory) allows for storing messages and then extracts the messages in a variable.</font></b>

In [5]:
from langchain.memory import ConversationBufferMemory

In [14]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True,
)

In [15]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [16]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [17]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. Is there anything else you would like to know or discuss?'

From the `Current converssation` section, the chatting history is kept and therefore LLM knows your name from previous discussion.

Let's take a look at how memory works.

In [20]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Andrew. Is there anything else you would like to know or discuss?


In [22]:
# load_memory_variables: Return history buffer.
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew. Is there anything else you would like to know or discuss?"}

Or you could feed in the chatting history this way:

In [23]:
memory = ConversationBufferMemory()

In [24]:
# Save context from this conversation to buffer.
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [25]:
print(memory.buffer)

Human: Hi
AI: What's up


In [26]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [27]:
# Add one more chatting record:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [28]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

<a id='sect2'></a>
## <b><font color='darkblue'>ConversationBufferWindowMemory</font></b>
<b><font size='3ptx'>[ConversationBufferWindowMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer_window.ConversationBufferWindowMemory.html) keeps a list of the interactions of the conversation over time. It only uses the last `K` interactions.</font></b>

<b>This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.</b> Let's first explore the basic functionality of this type of memory.

In [31]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) 

For `k=1`, it only remember the last chatting record.

In [32]:
# First chatting record
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

# Second chatting record
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [35]:
# The first chatting record will be dropped
print(memory.load_memory_variables({})['history'])

Human: Not much, just hanging
AI: Cool


In [36]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False)

In [37]:
_ = conversation.predict(input="Hi, my name is Andrew")  # Chatting record 1
_ = conversation.predict(input="What is 1+1?")           # chatting record 2

In [38]:
resp = conversation.predict(input="What is my name?")

In [40]:
# Because the chatting record 1 is flushed, LLM won't know the answer.
resp

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know?"

<a id='sect3'></a> 
## <b><font color='darkblue'>ConversationTokenBufferMemory</font></b> ([back](#sect0))
<b><font size='3ptx'>[ConversationTokenBufferMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.token_buffer.ConversationTokenBufferMemory.html) keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.</font></b>

In [41]:
from langchain.memory import ConversationTokenBufferMemory

llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

Because different LLM will have different way to measure the length of tokens, so we have to feed in the LLM object in this kind of memory. Let's prepare some chatting history for learning how it work:

In [42]:
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [47]:
chat_history = memory.load_memory_variables({})['history']
print(len(chat_history))
print(f'{chat_history}')

39
Human: Chatbots are what?
AI: Charming!


Let's enlarge the `max_token_limit=100`:

In [48]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)

In [49]:
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [50]:
chat_history = memory.load_memory_variables({})['history']
print(len(chat_history))
print(f'{chat_history}')

119
Human: AI is what?!
AI: Amazing!
Human: Backpropagation is what?
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


<a id='sect4'></a>
## <b><font color='darkblue'>ConversationSummaryMemory</font></b> ([back](#sect0))
<b><font size='3ptx'>[ConversationSummaryMemory](https://api.python.langchain.com/en/latest/memory/langchain.memory.summary.ConversationSummaryMemory.html) creates a summary of the conversation over time.</font></b>

<b>This can be useful for condensing information from the conversation over time. Conversation summary memory summarizes the conversation as it happens and stores the current summary in memory</b>. This memory can then be used to inject the summary of the conversation so far into a prompt/chain. This memory is most useful for longer conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

In [51]:
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

Let's create some chatting history for testing this kind of memory:

In [52]:
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [54]:
print(memory.load_memory_variables({})['history'])

System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's activities.


In [55]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True)

In [56]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's activities.
Human: What would be a good demo to show?
AI:

> Finished chain.


'For the morning meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal. This could include a live demonstration of the language processing capabilities, data encryption features, and any new integrations that have been implemented. Additionally, for the lunch meeting with the customer interested in AI, a demo highlighting the AI capabilities of our products and services would be impressive. This could involve showcasing machine learning algorithms, natural language processing tools, and any personalized recommendations based on user data. Let me know if you need help preparing any specific demos!'

In [57]:
print(memory.load_memory_variables({})['history'])

System: The human and AI exchange greetings and discuss the day's schedule. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of being prepared for the day's activities, suggesting demos for the meetings that showcase the latest features and updates on the LangChain project, as well as the AI capabilities of their products and services. The AI offers assistance in preparing specific demos if needed.


## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearning.ai - Langchain Ch1: Models, prompts and parsers](https://learn.deeplearning.ai/courses/langchain/lesson/2/models%2C-prompts-and-parsers)
* [Deeplearning.ai - Langchain Ch4: Chain](https://learn.deeplearning.ai/courses/langchain/lesson/4/chains)